In [2]:
from utils import MidiProcessor, prepare_data, array_to_midi, concat_all_midi_to_df, get_good_files,get_bad_files, get_files
from model import build_lstm
import numpy as np
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.utils import np_utils
from keras.models import load_model
import matplotlib.pyplot as plt

In [10]:
input_window_len = 32
pred_steps = 1

In [11]:
files = get_good_files()
df, tracks_len_list = concat_all_midi_to_df(files, return_tracks_len_list=True)

14 drum loops
25 percussion instruments
40018 32-notes


In [7]:
X_train, y_train, X_test, y_test, instruments_list = prepare_data(df, input_window_len=input_window_len, pred_steps=1, overlaps=0, train_test_split=0.1,
                                                                  tracks_len_list=tracks_len_list, max_instruments=10)

In [8]:
X_train

array([[[1, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 1, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 1, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 

In [9]:
n_feature = X_train.shape[2]
output_shape = y_train.shape[1]
model = build_lstm(input_window_len=input_window_len, n_feature=n_feature, output_shape=output_shape, dropout=0.5, neurons_per_layer=[32, 32, 32])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32, 32)            5504      
_________________________________________________________________
dropout (Dropout)            (None, 32, 32)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32, 512)           1116160   
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 512)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 32, 512)           2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 512)           0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0

In [11]:
es = EarlyStopping(monitor='val_loss', patience=20)
mc = ModelCheckpoint(filepath='./lstm_2nd_try_50_s.h5',
                     monitor='val_loss', verbose=1, save_best_only=True)
callbacks = [es, mc]

history = model.fit(X_train, y_train, epochs=50, callbacks=callbacks,
                    validation_split=0.1, verbose=1, shuffle=False)

Epoch 1/50
1002/1002 [==============================] - 377s 373ms/step - loss: 0.1475 - accuracy: 0.5962 - val_loss: 0.1476 - val_accuracy: 0.1617

Epoch 00001: val_loss improved from inf to 0.14764, saving model to ./lstm_2nd_try_50_s.h5
Epoch 2/50
1002/1002 [==============================] - 375s 374ms/step - loss: 0.1126 - accuracy: 0.6238 - val_loss: 0.1312 - val_accuracy: 0.1645

Epoch 00002: val_loss improved from 0.14764 to 0.13115, saving model to ./lstm_2nd_try_50_s.h5
Epoch 3/50
1002/1002 [==============================] - 375s 374ms/step - loss: 0.1012 - accuracy: 0.6288 - val_loss: 0.1137 - val_accuracy: 0.1867

Epoch 00003: val_loss improved from 0.13115 to 0.11369, saving model to ./lstm_2nd_try_50_s.h5
Epoch 4/50
1002/1002 [==============================] - 374s 374ms/step - loss: 0.0882 - accuracy: 0.5781 - val_loss: 0.0969 - val_accuracy: 0.2294

Epoch 00004: val_loss improved from 0.11369 to 0.09694, saving model to ./lstm_2nd_try_50_s.h5
Epoch 5/50
1002/1002 [======

In [8]:
model = load_model('./best_models/lstm_1nd_try_50_t.h5')

In [9]:
y_pred = (model.predict(X_test) > 0.2).astype("int32")

NameError: name 'X_test' is not defined

In [20]:
song1_lstm = array_to_midi(y_pred, instruments_list, 140)

In [21]:
song1_lstm.save('lstm_2nd_try_50_sev_02.mid')